In [ ]:
import torch
X = torch.tensor([
  [1,2],
  [2,4],
  [3,8]
])

In [ ]:
XX = X.repeat(3,1,1)
XX.shape

torch.Size([3, 3, 2])

In [ ]:
XX

tensor([[[1, 2],
         [2, 4],
         [3, 8]],

        [[1, 2],
         [2, 4],
         [3, 8]],

        [[1, 2],
         [2, 4],
         [3, 8]]])

In [ ]:
XXT = XX.transpose(0,1)

In [ ]:
deltaX = XXT - XX

In [ ]:
steps = XXT[None,:,:,:].repeat(4,1,1,1)

In [ ]:
steps.shape

torch.Size([4, 3, 3, 2])

In [ ]:
XXT

tensor([[[1., 2.],
         [1., 2.],
         [1., 2.]],

        [[2., 4.],
         [2., 4.],
         [2., 4.]],

        [[3., 8.],
         [3., 8.],
         [3., 8.]]])

In [ ]:
steps[2,:,:,:]

tensor([[[1., 2.],
         [1., 2.],
         [1., 2.]],

        [[2., 4.],
         [2., 4.],
         [2., 4.]],

        [[3., 8.],
         [3., 8.],
         [3., 8.]]])

In [ ]:
step_distances = deltaX[None,:,:,:].repeat(4,1,1,1)
step_distances.shape

torch.Size([4, 3, 3, 2])

In [ ]:
stepnums = torch.arange(4)[None,None,None,:].T.repeat(1,3,3,2)
stepnums.shape

torch.Size([4, 3, 3, 2])

In [ ]:
stepnums

tensor([[[[0, 0],
          [0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0],
          [0, 0]]],


        [[[1, 1],
          [1, 1],
          [1, 1]],

         [[1, 1],
          [1, 1],
          [1, 1]],

         [[1, 1],
          [1, 1],
          [1, 1]]],


        [[[2, 2],
          [2, 2],
          [2, 2]],

         [[2, 2],
          [2, 2],
          [2, 2]],

         [[2, 2],
          [2, 2],
          [2, 2]]],


        [[[3, 3],
          [3, 3],
          [3, 3]],

         [[3, 3],
          [3, 3],
          [3, 3]],

         [[3, 3],
          [3, 3],
          [3, 3]]]])

In [ ]:
steps

tensor([[[[1, 1, 1, 1],
          [2, 2, 2, 2]],

         [[1, 1, 1, 1],
          [2, 2, 2, 2]],

         [[1, 1, 1, 1],
          [2, 2, 2, 2]]],


        [[[2, 2, 2, 2],
          [4, 4, 4, 4]],

         [[2, 2, 2, 2],
          [4, 4, 4, 4]],

         [[2, 2, 2, 2],
          [4, 4, 4, 4]]],


        [[[3, 3, 3, 3],
          [8, 8, 8, 8]],

         [[3, 3, 3, 3],
          [8, 8, 8, 8]],

         [[3, 3, 3, 3],
          [8, 8, 8, 8]]]])

In [ ]:
steps_to_take = step_distances * stepnums

In [ ]:
pos = steps + steps_to_take
pos

tensor([[[[  1,   1,   1,   1],
          [  2,   2,   2,   2]],

         [[  1,   0,  -1,  -2],
          [  2,   0,  -2,  -4]],

         [[  1,  -1,  -3,  -5],
          [  2,  -4, -10, -16]]],


        [[[  2,   3,   4,   5],
          [  4,   6,   8,  10]],

         [[  2,   2,   2,   2],
          [  4,   4,   4,   4]],

         [[  2,   1,   0,  -1],
          [  4,   0,  -4,  -8]]],


        [[[  3,   5,   7,   9],
          [  8,  14,  20,  26]],

         [[  3,   4,   5,   6],
          [  8,  12,  16,  20]],

         [[  3,   3,   3,   3],
          [  8,   8,   8,   8]]]])

In [ ]:
pos[:,:,:,2]

tensor([[[  1,   2],
         [ -1,  -2],
         [ -3, -10]],

        [[  4,   8],
         [  2,   4],
         [  0,  -4]],

        [[  7,  20],
         [  5,  16],
         [  3,   8]]])

In [ ]:
pos_in = pos.transpose(2,3)

In [ ]:
pos_in.shape

torch.Size([3, 3, 4, 2])

In [ ]:
pos_in[:,:,0,:]

tensor([[[1, 2],
         [1, 2],
         [1, 2]],

        [[2, 4],
         [2, 4],
         [2, 4]],

        [[3, 8],
         [3, 8],
         [3, 8]]])

In [ ]:
from directed_graphs.flow_embedding import FlowEmbedder
from directed_graphs.datasets import SmallRandom, visualize_graph, ChainGraph
import torch_geometric
from torch_geometric.utils import to_networkx
dataset = ChainGraph()
data = dataset[0]
fe = FlowEmbedder(data, "euclidean",1)

/Users/adjourner/Projects/directed_graphs/directed_graphs/flow_embedding.py:57: UserWarning: Graph is not strongly connected. Embedding results are not guaranteed. Consider tweaking the constant for 'disconnected distance'.
  warnings.warn("Graph is not strongly connected. Embedding results are not guaranteed. Consider tweaking the constant for 'disconnected distance'.")


In [ ]:
pos_in = pos_in.float()

In [ ]:
flow_out = fe.flowfield(pos_in)

In [ ]:
flow_out = flow_out.transpose(2,3)

In [ ]:
flow_out.shape

torch.Size([3, 3, 2, 4])

In [ ]:
deltaX.shape

torch.Size([3, 3, 2])

In [ ]:
torch.tensordot(flow_out,deltaX,dims=2)

RuntimeError: contracted dimensions need to match, but first has size 2 in dim -2 and second has size 3 in dim 0

In [ ]:
a = torch.rand(3,3,2,4)
b = torch.rand(3,3,2)
c = torch.tensordot(a,b,dims=[[1,2],[0,1,2]])
c.shape

torch.Size([3, 4, 3])

In [ ]:
a[0,:,:,:,0]

tensor([[[0.9905, 0.2605, 0.6550],
         [0.1738, 0.8838, 0.5504],
         [0.6923, 0.0247, 0.5766]],

        [[0.3302, 0.1403, 0.8270],
         [0.7124, 0.4523, 0.8484],
         [0.6972, 0.2148, 0.3862]],

        [[0.5117, 0.3420, 0.0667],
         [0.1627, 0.5445, 0.0698],
         [0.0639, 0.1406, 0.7387]],

        [[0.3382, 0.0722, 0.4617],
         [0.7326, 0.1880, 0.1645],
         [0.1589, 0.3219, 0.6908]]])

In [ ]:
b[:,:,0]

tensor([[0.5923, 0.1850, 0.3997],
        [0.0681, 0.0649, 0.2206],
        [0.9665, 0.8905, 0.9769]])

In [ ]:
c[:,:,0]

tensor([[1.4605, 0.9869, 0.8526],
        [0.7624, 1.3999, 1.2721],
        [0.4276, 0.9280, 0.5488],
        [0.6807, 1.1729, 0.8598]])

In [ ]:
import torch
embedded_points = torch.tensor([
  [1,2],
  [2,4],
  [3,8]
]).float()

In [ ]:
embedded_points.type()

'torch.FloatTensor'

In [ ]:
def cost_normalizer(dot):
		return 3*torch.exp(dot)/((3 - 1) + torch.exp(dot))

In [ ]:
nnodes = 3
num_steps = 4
embedding_dimension = 2
embedding_D = torch.empty(nnodes,nnodes)
# Find matrix of delta x
XX = embedded_points.repeat(nnodes,1,1)
XXT = XX.transpose(0,1)
deltaX = XXT - XX
# Discretize the line between the points 
# This is an array with copies of the points
steps = XXT[None,:,:,:].repeat(num_steps,1,1,1)
# Now add to this the distance travelled in each step
stepnums = torch.arange(num_steps)[None,None,None,:].T.repeat(1,nnodes,nnodes,embedding_dimension)
step_distances = deltaX[None,:,:,:].repeat(num_steps,1,1,1) * stepnums
steps = steps + step_distances
# Evaluate the flows at each step
flows_per_step = fe.flowfield(steps)
# Take the dot product between the flows per step and the delta x directions
rdx = deltaX[None,:,:,:].repeat(num_steps,1,1,1)
dots_with_flow = (rdx*flows_per_step).sum(-1)
# Calculate cost based on dot products
embedding_D = cost_normalizer(dots_with_flow)


In [ ]:
dots_with_flow

tensor([[[ 0.0000, -0.1583, -0.3225],
         [ 0.2375,  0.0000, -0.3109],
         [ 0.3695,  0.1996,  0.0000]],

        [[ 0.0000, -0.1012,  0.2026],
         [ 0.2531,  0.0000, -0.5354],
         [ 0.0919,  0.0566,  0.0000]],

        [[ 0.0000,  0.0471,  0.2875],
         [ 0.2291,  0.0000, -0.0830],
         [-0.0706, -0.0461,  0.0000]],

        [[ 0.0000,  0.1327,  0.2686],
         [ 0.1918,  0.0000,  0.1812],
         [-0.1276, -0.0998,  0.0000]]], grad_fn=<SumBackward1>)

In [ ]:
dots_with_flow.sum(0)

tensor([[ 0.0000, -0.0797,  0.4361],
        [ 0.9116,  0.0000, -0.7481],
        [ 0.2633,  0.1103,  0.0000]], grad_fn=<SumBackward1>)

In [ ]:
embedding_D

tensor([[[1.0000, 0.8974, 0.7976],
         [1.1641, 1.0000, 0.8044],
         [1.2593, 1.1372, 1.0000]],

        [[1.0000, 0.9337, 1.1393],
         [1.1752, 1.0000, 0.6793],
         [1.0622, 1.0381, 1.0000]],

        [[1.0000, 1.0317, 1.1998],
         [1.1581, 1.0000, 0.9455],
         [0.9535, 0.9695, 1.0000]],

        [[1.0000, 1.0903, 1.1863],
         [1.1317, 1.0000, 1.1242],
         [0.9168, 0.9346, 1.0000]]], grad_fn=<DivBackward0>)

In [ ]:
flows_per_step.shape

torch.Size([4, 3, 3, 2])

In [ ]:
deltaX.shape

torch.Size([3, 3, 2])

In [ ]:
dot_products_with_flows = torch.tensordot(flows_per_step,deltaX,dims=[[2,3],[1,2]])
dot_products_with_flows.shape

torch.Size([4, 3, 3])

In [ ]:
dot_products_with_flows

tensor([[[-0.4808, -0.0059,  0.4867],
         [-0.7859, -0.0734,  0.8593],
         [-0.5393, -0.0297,  0.5691]],

        [[ 0.1013,  0.3581,  0.3420],
         [-1.1830, -0.2823,  0.9074],
         [-0.4626, -0.1262,  0.1485]],

        [[ 0.3346,  0.4114,  0.0528],
         [-0.4853,  0.1462,  0.8426],
         [-0.4087, -0.1822, -0.1166]],

        [[ 0.4013,  0.3991, -0.0928],
         [-0.0636,  0.3730,  0.7385],
         [-0.3806, -0.2018, -0.2273]]], grad_fn=<ReshapeAliasBackward0>)

In [ ]:
flows_per_step

tensor([[[[ 0.1524,  0.0030],
          [ 0.1524,  0.0030],
          [ 0.1524,  0.0030]],

         [[ 0.1641,  0.0367],
          [ 0.1641,  0.0367],
          [ 0.1641,  0.0367]],

         [[ 0.1401,  0.0149],
          [ 0.1401,  0.0149],
          [ 0.1401,  0.0149]]],


        [[[ 0.1524,  0.0030],
          [ 0.1830, -0.0409],
          [ 0.1943, -0.0985]],

         [[ 0.1629,  0.0451],
          [ 0.1641,  0.0367],
          [ 0.2849,  0.0626]],

         [[ 0.1282, -0.0274],
          [ 0.1298, -0.0183],
          [ 0.1401,  0.0149]]],


        [[[ 0.1524,  0.0030],
          [ 0.1754, -0.1113],
          [ 0.1844, -0.1094]],

         [[ 0.1557,  0.0367],
          [ 0.1641,  0.0367],
          [ 0.2467, -0.0409]],

         [[ 0.1228, -0.0527],
          [ 0.1246, -0.0427],
          [ 0.1401,  0.0149]]],


        [[[ 0.1524,  0.0030],
          [ 0.1499, -0.1413],
          [ 0.1853, -0.1065]],

         [[ 0.1480,  0.0219],
          [ 0.1641,  0.0367],
          [ 0.

In [ ]:
rdx = deltaX[None,:,:,:].repeat(4,1,1,1)
rdx.shape

torch.Size([4, 3, 3, 2])

In [ ]:
a = torch.rand(3,3,2)
b = torch.rand(3,3,2)

In [ ]:
a

tensor([[[0.3343, 0.4113],
         [0.2596, 0.6749],
         [0.7037, 0.1405]],

        [[0.4813, 0.7053],
         [0.2458, 0.9340],
         [0.6074, 0.0353]],

        [[0.5028, 0.9178],
         [0.8514, 0.8103],
         [0.5991, 0.8338]]])

In [ ]:
b

tensor([[[0.4312, 0.1273],
         [0.1215, 0.2547],
         [0.1485, 0.1774]],

        [[0.1105, 0.8145],
         [0.6118, 0.9752],
         [0.5687, 0.7572]],

        [[0.9022, 0.6650],
         [0.5273, 0.7363],
         [0.2966, 0.0496]]])

In [ ]:
c = torch.tensordot(a,b,dims=[[1,2],[1,2]])
c.shape

torch.Size([3, 3])

In [ ]:
(a*b).sum(dim=-1)

tensor([[0.1965, 0.2034, 0.1294],
        [0.6277, 1.0612, 0.3722],
        [1.0640, 1.0456, 0.2191]])